# **Q1(A)**

In [ ]:
!pip install pymongo

In [ ]:
!pip install geopandas

In [ ]:
!pip install pygeos

In [ ]:
import pandas as pd

In [ ]:
from pymongo import MongoClient
myClient = MongoClient('localhost', 27017)
myDB = myClient['hurricaneDB']

In [ ]:
hurricaneTable = myDB["hurricane"]

In [ ]:
hurricane_path = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2021-100522.txt'
raw_df = pd.read_csv(hurricane_path, header=None, sep="|")

In [ ]:
raw_df.count()

0    55437
dtype: int64

In [ ]:
storms = {}
header = None
for line in  raw_df[0]:
    tokens = line.split(',')
    if len(tokens) == 4:
        header = line
        storms[header] = []
    elif len(tokens) == 21:
        row = [token.strip() for token in tokens]
        storms[header].append(row)

In [ ]:
len(storms)

1936

In [ ]:
frames = []
#key example: AL011851, UNNAMED, 14,
for storm in storms:
    code, name, entries, blank= [record.strip() for record in storm.split(',')]
    basic_columns = {
        'BASIN' : code[:2],
        'ATCF_CY_NUMBER': code[2:4],
        'YEAR' :code[4:9],
        'NAME' : name,
        'NUM_BEST_TRACK_ENTRIES': entries
    }

    dataset = pd.DataFrame(storms[storm], columns=[
                                                    'DATE',
                                                    'TIME_UTC',
                                                    'POINT_TYPE',
                                                    'STATUS',
                                                    'LATITUDE',
                                                    'LONGITUDE',
                                                    'MAX_WINDSPEED_KT',
                                                    'MIN_PRESURE_MB',
                                                    'NE_34KT',
                                                    'SE_34KT',
                                                    'NW_34_KT',
                                                    'SW_34_KT',
                                                    'NE_50KT',
                                                    'SE_50KT',
                                                    'NW_50_KT',
                                                    'SW_50_KT',
                                                    'NE_64KT',
                                                    'SE_64KT',
                                                    'NW_64_KT',
                                                    'SW_64_KT',
                                                    'RADIUS'

                                                   ]
                          )
    for column in basic_columns:
        dataset[column] = basic_columns[column]
    frames.append(dataset)
final_df = pd.concat(frames)

In [ ]:
final_df.head()

,DATE,TIME_UTC,POINT_TYPE,STATUS,LATITUDE,LONGITUDE,MAX_WINDSPEED_KT,MIN_PRESURE_MB,NE_34KT,SE_34KT,...,NE_64KT,SE_64KT,NW_64_KT,SW_64_KT,RADIUS,BASIN,ATCF_CY_NUMBER,YEAR,NAME,NUM_BEST_TRACK_ENTRIES
0,18510625,0000,,HU,28.0N,94.8W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
1,18510625,0600,,HU,28.0N,95.4W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
2,18510625,1200,,HU,28.0N,96.0W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
3,18510625,1800,,HU,28.1N,96.5W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
4,18510625,2100,L,HU,28.2N,96.8W,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14


# **Q1(B)**

In [ ]:
for i,df in enumerate(final_df['LONGITUDE']):
  if "W" in df:
    final_df['LONGITUDE'].iloc[i] = float(df.strip('W'))*-1
  elif "E" in df:
    final_df['LONGITUDE'].iloc[i] = float(df.strip('E'))

for i,df in enumerate(final_df['LATITUDE']):
  if "N" in df:
    final_df['LATITUDE'].iloc[i] = float(df.strip('N'))
  elif "S" in df:
    final_df['LATITUDE'].iloc[i] = float(df.strip('S'))*-1

In [ ]:
final_df.head()

,DATE,TIME_UTC,POINT_TYPE,STATUS,LATITUDE,LONGITUDE,MAX_WINDSPEED_KT,MIN_PRESURE_MB,NE_34KT,SE_34KT,...,NE_64KT,SE_64KT,NW_64_KT,SW_64_KT,RADIUS,BASIN,ATCF_CY_NUMBER,YEAR,NAME,NUM_BEST_TRACK_ENTRIES
0,18510625,0000,,HU,28.0,-94.8,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
1,18510625,0600,,HU,28.0,-95.4,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
2,18510625,1200,,HU,28.0,-96.0,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
3,18510625,1800,,HU,28.1,-96.5,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14
4,18510625,2100,L,HU,28.2,-96.8,80,-999,-999,-999,...,-999,-999,-999,-999,-999,AL,01,1851,UNNAMED,14


In [ ]:
final_df['LONGITUDE'] = final_df['LONGITUDE'].astype(float)
final_df['LATITUDE'] = final_df['LATITUDE'].astype(float)

In [ ]:
final_df["DATE"] = pd.to_datetime(final_df["DATE"],format='%Y%m%d')
final_df['TIME_UTC'] =pd.to_datetime(final_df.TIME_UTC, format='%H%M').dt.strftime('%H:%M')

# **Q1(C)**

In [ ]:
final_df['MAX_WINDSPEED_KT'] = final_df['MAX_WINDSPEED_KT'].astype('int')
final_df['MIN_PRESURE_MB'] = final_df['MIN_PRESURE_MB'].astype('int')
final_df['NUM_BEST_TRACK_ENTRIES'] = final_df['NUM_BEST_TRACK_ENTRIES'].astype('int')
final_df['NE_34KT'] = final_df['NE_34KT'].astype('int')
final_df['SE_34KT'] = final_df['SE_34KT'].astype('int')
final_df['NW_34_KT'] = final_df['NW_34_KT'].astype('int')
final_df['SW_34_KT'] = final_df['SW_34_KT'].astype('int')
final_df['NE_50KT'] = final_df['NE_50KT'].astype('int')
final_df['SE_50KT'] = final_df['SE_50KT'].astype('int')
final_df['NW_50_KT'] = final_df['NW_50_KT'].astype('int')
final_df['SW_50_KT'] = final_df['SW_50_KT'].astype('int')
final_df['NE_64KT'] = final_df['NE_64KT'].astype('int')
final_df['SE_64KT'] = final_df['SE_64KT'].astype('int')
final_df['NW_64_KT'] = final_df['NW_64_KT'].astype('int')
final_df['SW_64_KT'] = final_df['SW_64_KT'].astype('int')
final_df['RADIUS'] = final_df['RADIUS'].astype('int')

In [ ]:
final_df.dtypes

DATE                      datetime64[ns]
TIME_UTC                          object
POINT_TYPE                        object
STATUS                            object
LATITUDE                         float64
LONGITUDE                        float64
MAX_WINDSPEED_KT                   int32
MIN_PRESURE_MB                     int32
NE_34KT                            int32
SE_34KT                            int32
NW_34_KT                           int32
SW_34_KT                           int32
NE_50KT                            int32
SE_50KT                            int32
NW_50_KT                           int32
SW_50_KT                           int32
NE_64KT                            int32
SE_64KT                            int32
NW_64_KT                           int32
SW_64_KT                           int32
RADIUS                             int32
BASIN                             object
ATCF_CY_NUMBER                    object
YEAR                              object
NAME            

## **Q1(D)**

In [ ]:
final_df['PRIMARY_KEY'] = final_df['DATE'].astype(str) + '_' + \
                          final_df['TIME_UTC'].astype(str) + '_' + final_df['LATITUDE'].astype(str)+'_' +\
                          final_df['LONGITUDE'].astype(str)

In [ ]:
final_df.head()

,DATE,TIME_UTC,POINT_TYPE,STATUS,LATITUDE,LONGITUDE,MAX_WINDSPEED_KT,MIN_PRESURE_MB,NE_34KT,SE_34KT,...,SE_64KT,NW_64_KT,SW_64_KT,RADIUS,BASIN,ATCF_CY_NUMBER,YEAR,NAME,NUM_BEST_TRACK_ENTRIES,PRIMARY_KEY
0,1851-06-25,00:00,,HU,28.0,-94.8,80,-999,-999,-999,...,-999,-999,-999,-999,AL,01,1851,UNNAMED,14,1851-06-25_00:00_28.0_-94.8
1,1851-06-25,06:00,,HU,28.0,-95.4,80,-999,-999,-999,...,-999,-999,-999,-999,AL,01,1851,UNNAMED,14,1851-06-25_06:00_28.0_-95.4
2,1851-06-25,12:00,,HU,28.0,-96.0,80,-999,-999,-999,...,-999,-999,-999,-999,AL,01,1851,UNNAMED,14,1851-06-25_12:00_28.0_-96.0
3,1851-06-25,18:00,,HU,28.1,-96.5,80,-999,-999,-999,...,-999,-999,-999,-999,AL,01,1851,UNNAMED,14,1851-06-25_18:00_28.1_-96.5
4,1851-06-25,21:00,L,HU,28.2,-96.8,80,-999,-999,-999,...,-999,-999,-999,-999,AL,01,1851,UNNAMED,14,1851-06-25_21:00_28.2_-96.8


## **Q1(E)**

In [ ]:
with open('hurricane_updated.json', 'w') as f:
    f.write(final_df.to_json(orient = 'records', compression = 'infer', index = 'false'))

# **Q1(F)**

In [ ]:
import json
items=[]
with open('hurricane_updated.json') as f:
    items=json.load(f)

In [ ]:
Items= []
for item in items:
    hurr_item={}
    hurr_item["PRIMARY_KEY"]= item["PRIMARY_KEY"]
    hurr_item["YEAR"]= item["YEAR"]
    del item["PRIMARY_KEY"]
    del item["YEAR"]
    hurr_item["INFO"]=item
    Items.append(hurr_item)

In [ ]:
Items[0]

{'PRIMARY_KEY': '1851-06-25_00:00_28.0_-94.8',
 'YEAR': '1851',
 'INFO': {'DATE': -3740169600000,
  'TIME_UTC': '00:00',
  'POINT_TYPE': '',
  'STATUS': 'HU',
  'LATITUDE': 28.0,
  'LONGITUDE': -94.8,
  'MAX_WINDSPEED_KT': 80,
  'MIN_PRESURE_MB': -999,
  'NE_34KT': -999,
  'SE_34KT': -999,
  'NW_34_KT': -999,
  'SW_34_KT': -999,
  'NE_50KT': -999,
  'SE_50KT': -999,
  'NW_50_KT': -999,
  'SW_50_KT': -999,
  'NE_64KT': -999,
  'SE_64KT': -999,
  'NW_64_KT': -999,
  'SW_64_KT': -999,
  'RADIUS': -999,
  'BASIN': 'AL',
  'ATCF_CY_NUMBER': '01',
  'NAME': 'UNNAMED',
  'NUM_BEST_TRACK_ENTRIES': 14}}

In [ ]:
hurricaneTable.insert_many(Items)

## **Q2(A)**

In [ ]:
data = hurricaneTable.find({})

In [ ]:
count=0
for i in data:
    count+=1
print(count)

53501


## **Q2(B)**

In [ ]:
hu_data = hurricaneTable.find({'INFO.STATUS': {"$eq": 'HU'}, 'YEAR' : {"$gte": "2020", "$lte": "2021"}})

In [ ]:
count=0
for i in hu_data:
    count+=1
print(count)

273


## **Q2(C)**

In [ ]:
max_wind_speed = hurricaneTable.find({'INFO.MAX_WINDSPEED_KT' : {"$gte": 112.97, "$lte": 135.56}, 'YEAR' : {"$gte": "2000"}})

In [ ]:
response_list=[]
for i in max_wind_speed:
    if isinstance(i, list):
        for j in i:
            response_list.append({
                "YEAR": j['YEAR'],
                "NAME": j['INFO']["NAME"],

            })
    else:
        response_list.append({
                "YEAR": i['YEAR'],
                "NAME": i['INFO']["NAME"],
            })

In [ ]:
new_max_speed_list = sorted(response_list, key=lambda x: x["YEAR"], reverse=False)

In [ ]:
print("{}\t{}".format('YEAR', 'NAME'))
dataSet=[]
for item in new_max_speed_list:
    if item['YEAR']+item['NAME'] not in dataSet:
        print("{}\t{}".format(item['YEAR'], item['NAME']))
        dataSet.append(item['YEAR']+item['NAME'])



YEAR	NAME
2000	ISAAC
2000	KEITH
2001	IRIS
2001	MICHELLE
2002	LILI
2003	FABIAN
2003	ISABEL
2004	CHARLEY
2004	FRANCES
2004	IVAN
2004	KARL
2005	DENNIS
2005	EMILY
2005	KATRINA
2005	RITA
2005	WILMA
2007	DEAN
2007	FELIX
2008	GUSTAV
2008	IKE
2008	OMAR
2008	PALOMA
2009	BILL
2010	DANIELLE
2010	EARL
2010	IGOR
2010	JULIA
2011	KATIA
2011	OPHELIA
2014	GONZALO
2015	JOAQUIN
2016	MATTHEW
2016	NICOLE
2017	HARVEY
2017	IRMA
2017	JOSE
2017	MARIA
2018	FLORENCE
2018	MICHAEL
2019	DORIAN
2019	LORENZO
2020	LAURA
2020	TEDDY
2020	DELTA
2020	ETA
2020	IOTA
2021	IDA
2021	SAM


## **Q2(D)**

In [ ]:
gilbert_data = hurricaneTable.find({'INFO.NAME' : {"$eq": "GILBERT"}})

In [ ]:
response_list=[]
for i in gilbert_data:
    if isinstance(i, list):
        for j in i:
            response_list.append({
                "YEAR": j['YEAR'],
                "NAME": j['INFO']["NAME"],
                "MAX_WINDSPEED_KT":j['INFO']["MAX_WINDSPEED_KT"],

            })
            # print(i1['INFO'])
    else:
        response_list.append({
                "YEAR": i['YEAR'],
                "NAME": i['INFO']["NAME"],
                 "MAX_WINDSPEED_KT":i['INFO']["MAX_WINDSPEED_KT"],

            })

In [ ]:
new_gilbert_data = sorted(response_list, key=lambda x: x["MAX_WINDSPEED_KT"], reverse=True)

In [ ]:
print("{}\t{}".format('YEAR', 'Maximum Sustained Winds '))
print("{}\t{}".format(new_gilbert_data[0]['YEAR'], new_gilbert_data[0]['MAX_WINDSPEED_KT']))

YEAR	Maximum Sustained Winds 
1988	160
